In [1]:
import math
import numpy as np
import h5py
import scipy
import cv2
from scipy import ndimage
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.python.framework import ops
from matplotlib.pyplot import imshow
from preprocessing import *
import pickle

%matplotlib inline
np.random.seed(1)

## Create Placeholders

In [2]:
def create_placeholders_for_training(n_H0, n_W0, n_C0):
    # n_H0 -- scalar, height of an input image
    # n_W0 -- scalar, width of an input image
    # n_C0 -- scalar, number of channels of the input
    # Returns
    # X,Y,Z -- placeholder for the data input, of shape [None, n_H0, n_W0, n_C0] and dtype "float"
    X = tf.placeholder(tf.float32, shape=(None,n_H0,n_W0,n_C0))
    Y = tf.placeholder(tf.float32, shape=(None,n_H0,n_W0,n_C0))
    Z = tf.placeholder(tf.float32, shape=(None,n_H0,n_W0,n_C0))
    return X, Y, Z
    

## Initialize Parameters

In [3]:
def init_params(n_C0):
    tf.set_random_seed(1)
    conv1 = tf.get_variable("conv1", [7,7,n_C0,64], initializer = tf.contrib.layers.xavier_initializer(seed = 1))
    conv2a= tf.get_variable("conv2a", [1,1,64,64], initializer = tf.contrib.layers.xavier_initializer(seed = 1))
    conv2 = tf.get_variable("conv2", [3,3,64,192], initializer = tf.contrib.layers.xavier_initializer(seed = 1))
    conv3a= tf.get_variable("conv3a", [1,1,192,192], initializer = tf.contrib.layers.xavier_initializer(seed = 1))
    conv3 = tf.get_variable("conv3", [3,3,192,384], initializer = tf.contrib.layers.xavier_initializer(seed = 1))
    conv4a= tf.get_variable("conv4a", [1,1,384,384], initializer = tf.contrib.layers.xavier_initializer(seed = 1))
    conv4 = tf.get_variable("conv4", [3,3,384,256], initializer = tf.contrib.layers.xavier_initializer(seed = 1))
    conv5a= tf.get_variable("conv5a", [1,1,256,256], initializer = tf.contrib.layers.xavier_initializer(seed = 1))
    conv5 = tf.get_variable("conv5", [3,3,256,256], initializer = tf.contrib.layers.xavier_initializer(seed = 1))
    conv6a= tf.get_variable("conv6a", [1,1,256,256], initializer = tf.contrib.layers.xavier_initializer(seed = 1))
    conv6 = tf.get_variable("conv6", [3,3,256,256], initializer = tf.contrib.layers.xavier_initializer(seed = 1))
    #fc1 = tf.get_variable("fc1", [7,7,,64], initializer = tf.contrib.layers.xavier_initializer(seed = 1))
    #fc2   = tf.get_variable("fc2", [7,7,,64], initializer = tf.contrib.layers.xavier_initializer(seed = 1))
    #fc7128= tf.get_variable("fc7128", [7,7,,64], initializer = tf.contrib.layers.xavier_initializer(seed = 1))
    parameters = {"conv1": conv1,
                  "conv2a": conv2a,
                  "conv2": conv2,
                  "conv3a": conv3a,
                  "conv3": conv3,
                  "conv4a": conv4a,
                  "conv4": conv4,
                  "conv5a": conv5a,
                  "conv5": conv5,
                  "conv6a": conv6a,
                  "conv6": conv6,
                  #"fc1": fc1,
                  #"fc2": fc2,
                  #"fc7128": fc7128,
                  }
    return parameters

## Forward Propagation

In [36]:
def forward_prop(parameters,x):
    
    conv1 = parameters['conv1']
    conv2a = parameters['conv2a']
    conv2 = parameters['conv2']
    conv3a = parameters['conv3a']
    conv3 = parameters['conv3']
    conv4a = parameters['conv4a']
    conv4 = parameters['conv4']
    conv5a = parameters['conv5a']
    conv5 = parameters['conv5']
    conv6a = parameters['conv6a']
    conv6 = parameters['conv6']
    
    #Conv 1
    Z1 = tf.nn.conv2d(x,conv1, strides = [1,2,2,1], padding = 'VALID')
    with tf.variable_scope("b1") as scope:
        B1 = tf.layers.batch_normalization(Z1,axis=1, epsilon=0.00001,reuse = tf.AUTO_REUSE)
        A1 = tf.nn.relu(B1)
        P1 = tf.nn.max_pool(A1, ksize = [1,3,3,1], strides = [1,2,2,1], padding = 'VALID')
        N1 = tf.nn.lrn(P1)
    
    #Conv 2A
    Z2a = tf.nn.conv2d(N1,conv2a, strides = [1,1,1,1], padding = 'SAME')
    A2a = tf.nn.relu(Z2a)
   
    #Conv 2
    Z2 = tf.nn.conv2d(A2a,conv2, strides = [1,1,1,1], padding = 'SAME')
    with tf.variable_scope("b2") as scope:
        B2 = tf.layers.batch_normalization(Z2,axis=1, epsilon=0.00001,reuse = tf.AUTO_REUSE)
        A2 = tf.nn.relu(B2)
        P2 = tf.nn.max_pool(A2, ksize = [1,3,3,1], strides = [1,2,2,1], padding = 'VALID')
        N2 = tf.nn.lrn(P2)
     
    #Conv 3A
    Z3a = tf.nn.conv2d(N2,conv3a, strides = [1,1,1,1], padding = 'SAME')
    A3a = tf.nn.relu(Z3a)  
    
    #Conv 3
    Z3 = tf.nn.conv2d(A3a,conv3, strides = [1,1,1,1], padding = 'SAME')
    with tf.variable_scope("b3") as scope:
        B3 = tf.layers.batch_normalization(Z3,axis=1, epsilon=0.00001,reuse = tf.AUTO_REUSE)
        A3 = tf.nn.relu(B3)
        P3 = tf.nn.max_pool(A3, ksize = [1,3,3,1], strides = [1,2,2,1], padding = 'VALID')
        N3 = tf.nn.lrn(P3)
    
    #Conv 4a
    Z4a = tf.nn.conv2d(N3,conv4a, strides = [1,1,1,1], padding = 'SAME')
    A4a = tf.nn.relu(Z4a)
    
    #Conv 4
    Z4 = tf.nn.conv2d(A4a,conv4, strides = [1,1,1,1], padding = 'SAME')
    with tf.variable_scope("b4") as scope:
        B4 = tf.layers.batch_normalization(Z4,axis=1, epsilon=0.00001,reuse = tf.AUTO_REUSE)
        A4 = tf.nn.relu(B4)
        N4 = tf.nn.lrn(A4)
    
    #Conv 5a
    Z5a = tf.nn.conv2d(N4,conv5a, strides = [1,1,1,1], padding = 'SAME')
    A5a = tf.nn.relu(Z5a)
    
    #Conv 5
    Z5 = tf.nn.conv2d(A5a,conv5, strides = [1,1,1,1], padding = 'SAME')
    with tf.variable_scope("b5") as scope:
        B5 = tf.layers.batch_normalization(Z5,axis=1, epsilon=0.00001,reuse = tf.AUTO_REUSE)
        A5 = tf.nn.relu(B5)
        N5 = tf.nn.lrn(A5)
    
    #Conv 6a
    Z6a = tf.nn.conv2d(N5,conv6a, strides = [1,1,1,1], padding = 'SAME')
    A6a = tf.nn.relu(Z6a)
    
    #Conv 6
    Z6 = tf.nn.conv2d(A6a,conv6, strides = [1,1,1,1], padding = 'SAME')
    with tf.variable_scope("b6") as scope:
        B6 = tf.layers.batch_normalization(Z6,axis=1, epsilon=0.00001,reuse = tf.AUTO_REUSE)
        A6 = tf.nn.relu(B6)
        P6 = tf.nn.max_pool(A6, ksize = [1,3,3,1], strides = [1,2,2,1], padding = 'VALID')
    
    #Flattening
    P6F = tf.contrib.layers.flatten(P6)
    
    #FC 1
    with tf.variable_scope("fc1") as scope:
        Z_FC1 = tf.contrib.layers.fully_connected(P6F,32*256,activation_fn=None,reuse = tf.AUTO_REUSE,scope = tf.get_variable_scope())
        A_FC1 = tf.nn.relu(Z_FC1)
    #Maxout
    #M_FC1 = tf.contrib.layers.maxout(A_FC1,32*128)
    
    #FC_2
    with tf.variable_scope("fc2") as scope:
        Z_FC2 = tf.contrib.layers.fully_connected(A_FC1,32*256,activation_fn=None,reuse = tf.AUTO_REUSE,scope = tf.get_variable_scope())
        A_FC2 = tf.nn.relu(Z_FC2)

    #Maxout
    #M_FC2 = tf.contrib.layers.maxout(A_FC2,32*128)
    
    #FC_7128
    with tf.variable_scope("fc3") as scope:
        Z_FC7 = tf.contrib.layers.fully_connected(A_FC2,128,activation_fn=None,reuse = tf.AUTO_REUSE,scope = tf.get_variable_scope())
        A_FC7 = tf.nn.relu(Z_FC7)

    #l2 Normalization
    embeddings = tf.nn.l2_normalize(A_FC7)
    
    return embeddings


## Defining the Loss Function

In [5]:
def triplet_loss_debug(y_pred, alpha = 0.5):
    
    anchor, positive, negative = y_pred[0], y_pred[1], y_pred[2]
   
    pos_dist = tf.reduce_sum(tf.square(tf.subtract(anchor,positive)))
    
    pos_dist2 = tf.Print(pos_dist, [pos_dist], "pos_dist ")
   
    neg_dist = tf.reduce_sum(tf.square(tf.subtract(anchor,negative)))
    
    neg_dist2 = tf.Print(neg_dist, [neg_dist], "neg_dist ")
    
    basic_loss = tf.add(tf.subtract(pos_dist2, neg_dist2) , alpha)
    basic_loss2 = tf.Print(basic_loss, [basic_loss], "basic loss: ")
    
    loss = tf.reduce_sum(tf.maximum(basic_loss2,0.0))
    
    loss2 = tf.Print(loss, [loss], "loss ")
    
    return loss2

def triplet_loss(y_pred, alpha = 0.5):
    anchor, positive, negative = y_pred[0], y_pred[1], y_pred[2]
    
    pos_dist = tf.reduce_sum(tf.square(tf.subtract(anchor,positive)),axis = -1)
   
    neg_dist = tf.reduce_sum(tf.square(tf.subtract(anchor,negative)),axis = -1)
    
    basic_loss = tf.add(tf.subtract(pos_dist, neg_dist) , alpha)
    
    loss = tf.reduce_sum(tf.maximum(basic_loss,0.0))
    
    return loss

## Setting up Data PreProcessing

In [14]:
def loadSampleData():
    fname = "./SampleDataset/1a.jpeg"
    image = cv2.imread(fname)
    anchor = np.expand_dims(cv2.resize(image, (220,220),interpolation = cv2.INTER_AREA),axis = 0)
    
    fname = "./SampleDataset/1b.jpeg"
    image = cv2.imread(fname)
    positive = np.expand_dims(cv2.resize(image, (220,220),interpolation = cv2.INTER_AREA),axis = 0)
    
    fname = "./SampleDataset/1c.jpeg"
    image = cv2.imread(fname)
    
    negative = np.expand_dims(cv2.resize(image, (220,220),interpolation = cv2.INTER_AREA),axis = 0)
    #print(negative.shape)
    
    return anchor,positive,negative
    #return 0,0,0

#anchor,positive,negative = loadSampleData()
def getLoaderInstance(batchSize = 64, batches = 15):
    loaderInstance = DataReader(batchSize,batches)
    return loaderInstance

def loadDataBatch(loaderInstance):
    tempList = loaderInstance.getData()
    anchor, positive, negative = (np.array(tempList[0]),np.array(tempList[1]),np.array(tempList[2]))
    #print(anchor.shape)
    return anchor,positive,negative

#anchor,positive,negative = loadDataBatch()
    

## Instantiating the Loader to start Preprocessing

In [15]:
batchSize = 64
batches = 40
dataLoaderInstance = getLoaderInstance(batchSize, batches)


NameError: name 'DataReader' is not defined

## Preprocessing and Caching

In [8]:
anchor = np.empty((batchSize,220,220,3))
positive = np.empty((batchSize,220,220,3))
negative = np.empty((batchSize,220,220,3))
for i in range(batches):
    anchor,positive,negative = loadDataBatch(dataLoaderInstance)
    with open('./cache/inputs'+str(i)+'.pkl', 'wb') as f:  # Python 3: open(..., 'wb')
        pickle.dump([anchor, positive, negative], f, 0)
    if(i%10==0):
        print(i)
        
print(anchor.shape)

0
10
20
30
(64, 220, 220, 3)


## Caching the loaded Data

In [ ]:
# with open('inputs6415.pkl', 'wb') as f:  # Python 3: open(..., 'wb')
#     pickle.dump([anchor, positive, negative], f)
# imshow(positive[1])
# import cv2
# cv2.imwrite("image.jpg", negative[0])

## Restoring the cache

In [6]:
with open('../inputs6415.pkl','rb') as f:  # Python 3: open(..., 'rb')
    anchor,positive,negative = pickle.load(f)
print(anchor.shape)

(64, 220, 220, 3)


In [16]:
def loadCache(iter):
    with open('./cache/inputs'+str(iter)+'.pkl','rb') as f:  # Python 3: open(..., 'rb')
        anchor,positive,negative = pickle.load(f)
    return anchor,positive,negative


## Setting up the Tensorflow Graph

In [37]:
tf.reset_default_graph()
with tf.variable_scope("FaceNet", reuse=tf.AUTO_REUSE):
    x,y,z = create_placeholders_for_training(220,220,3)
    params = init_params(3)
    preds1 = forward_prop(params,x)
    tf.get_variable_scope().reuse_variables()
    preds2 = forward_prop(params,y)
    tf.get_variable_scope().reuse_variables()
    preds3 = forward_prop(params,z)

loss = triplet_loss([preds1,preds2,preds3],0.5)
optim = tf.train.AdamOptimizer(0.0001,name = 'optim').minimize(loss)

init  = tf.global_variables_initializer()


## Training the Siamese Network

In [39]:
saver = tf.train.Saver()
with tf.Session() as sess:
    sess.run(init)
    epochs = 4
    #saver.restore(sess, './faceNet3')
    train_cache_file = './faceNet31.meta'
    if os.path.exists(train_cache_file):
        saver.restore(sess, './faceNet31')
        print("Restoring Model")
    else:
        print("No Saved Model Found. Starting training from scratch.")
    for epoch in range(epochs):
        avgCost = 0
        iters = 7
        for i in range(iters):
            #anchor,positive,negative = loadCache(i)
            curr_cost, _  = sess.run([loss, optim],feed_dict = {x:anchor[i*8:i*8+7],y:positive[i*8:i*8+7],z:negative[i*8:i*8+7]})
            avgCost+=curr_cost
            if(i%2==0):
                print("i:"+str(i)+", loss = "+str(curr_cost))
            #print(embed1)
            #print(embed2)
        avgCost/=iters
        print("Avg Loss :"+str(avgCost))
    
    saver.save(sess, './faceNet31')
    print("Model Saved to disk")


No Saved Model Found. Starting training from scratch.


InternalError: The CPU implementation of FusedBatchNorm only supports NHWC tensor format for now.
	 [[Node: FaceNet/b1_2/batch_normalization/FusedBatchNorm = FusedBatchNorm[T=DT_FLOAT, data_format="NCHW", epsilon=1.001e-05, is_training=false, _device="/job:localhost/replica:0/task:0/device:CPU:0"](FaceNet/Conv2D_22, FaceNet/b1/batch_normalization/gamma/read, FaceNet/b1/batch_normalization/beta/read, FaceNet/b1/batch_normalization/moving_mean/read, FaceNet/b1/batch_normalization/moving_variance/read)]]

Caused by op 'FaceNet/b1_2/batch_normalization/FusedBatchNorm', defined at:
  File "C:\ProgramData\Anaconda3\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\ProgramData\Anaconda3\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "C:\ProgramData\Anaconda3\lib\site-packages\traitlets\config\application.py", 

In [ ]:
def getDistanceBetweenEmbeddings(embedding1, embedding2):
    dist = np.sum(np.square(np.subtract(embedding1,embedding2)))
    return dist

# anchor = tf.placeholder(tf.float32, shape=(None,220,220,3))
# positive = tf.placeholder(tf.float32, shape=(None,220,220,3))
# pos_dist2 = tf.reduce_sum(tf.square(tf.subtract(anchor,positive)))

emb1 = sess.run(preds1,feed_dict = {x:np.expand_dims(anchor[16],axis =0)})
emb2 = sess.run(preds1,feed_dict = {x:np.expand_dims(negative[16],axis =0)})

#tfdist = sess.run(pos_dist2,feed_dict = {anchor:np.expand_dims(anchor[16],axis =0),positive:np.expand_dims(positive[16],axis =0)})

#emb3 = sess.run(preds1,feed_dict = {x:np.expand_dims(anchor[16],axis =0)})
emb4 = sess.run(preds1,feed_dict = {x:np.expand_dims(positive[16],axis =0)})

print(emb1)
print(emb4)
dist = getDistanceBetweenEmbeddings(emb1,emb2)
print(dist)
dist2 = getDistanceBetweenEmbeddings(emb1,emb4)
print(dist2)
#print(tfdist)

## Saving the Model

In [ ]:
saver = tf.train.Saver()
saver.save(sess, './my-firstmodel1')

## Close the Session

In [15]:
#sess.close()